In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, SelectKBest, f_regression

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx # загружаем данные

In [3]:
data = pd.read_excel('data/data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [4]:
data = data.drop(['condition', 'title_status', 'transmission', 'drive', 'size'], axis=1)
data.dropna(inplace=True)

In [5]:
y = data['price']
x = data.drop(['price'], axis=1)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [7]:
estimator = linear_model.LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1) # выделяем три столбца-признака для обучения, выбранные RFE
selector = selector.fit(x_train, y_train)
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [8]:
selector = SelectKBest(f_regression, k=3) # выделяем три столбца-признака для обучения, выбранные SelectKBest
selector.fit(x_train, y_train)
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [9]:
y = data['price']
x = data[['year', 'cylinders', 'lat']]

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [11]:
model = linear_model.LinearRegression()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
print(round(mean_absolute_error(y_test, y_predict))) # считаем метрику на первых трёх признаках

5224


In [12]:
y = data['price']
x = data[['year', 'cylinders', 'odometer']]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [14]:
model = linear_model.LinearRegression()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
print(round(mean_absolute_error(y_test, y_predict))) # считаем метрику на вторых трёх признаках

4921


на тестовой выборке наилучший результат показал метод отбора признаков SelectKBest. средняя абсолютная ошибка на признаках, отобранных SelectKBest составила 4921, а на признаках, отобранных RFE только 5224